In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
combinations(8, [4,4]).shape

(35, 2, 4)

In [4]:
combinations(8, [2,2,2,2]).shape

(105, 4, 2)

In [5]:
array = np.random.exponential(size=int(18e6))
bins = np.linspace(0, 10, 30)

In [6]:
%%timeit -o
np.histogram(array, bins)

1.79 s ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1.79 s ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [7]:
%%timeit -o
numba_unweighted_histo(array, bins)

214 ms ± 7.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 214 ms ± 7.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [8]:
%%timeit -o
jax_unweighted_histo(array, bins)

NameError: name 'jax_unweighted_histo' is not defined